In [1]:
import sqlalchemy
import pandas as pd
from sqlalchemy import create_engine, text, inspect
from sqlalchemy import Column, Integer, String, Float, ForeignKey,Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from uuid import uuid4
import datetime as dt
Base = declarative_base()

C:\Users\ddram\AppData\Local\Temp\ipykernel_1968\977354949.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [2]:
def tocsv(symbol):
    return pd.read_csv(f'../../collected_data/csvs/{symbol}-USD.csv')

In [3]:
def plot(symbol):
    tocsv(symbol)[["Date","Open"]].plot()


In [4]:
class Coin(Base):
    __tablename__= "coin_names"
    coin_name = Column(String(255),primary_key=True)

In [5]:
class Entry(Base):
    __tablename__ = "crypto_history"
    _id= Column(Integer,primary_key=True)
    coin_name = Column(String(255),ForeignKey(Coin.coin_name))
    date = Column(Date())
    open = Column(Float())
    high = Column(Float())
    low = Column(Float())
    close = Column(Float())
    volume = Column(Integer())
    

In [6]:
def entry(ind,coin_symbol,df):
    return Entry(coin_name = coin_symbol,date =dt.datetime.strptime(df.iloc[ind]["Date"],"%Y-%m-%d"),open =df.iloc[ind]["Open"],high =df.iloc[ind]["High"],low =df.iloc[ind]["Low"],close =df.iloc[ind]["Close"],volume =df.iloc[ind]["Volume"])

In [7]:
def add_df_to_table_please(session,fx,coin_symbol):
    df = tocsv(coin_symbol)
    n_rows = len(df.index)
    for i in range(n_rows):
        thing = entry(i,coin_symbol,df)
        session.add(thing)
    session.commit()

In [8]:
engine = create_engine('sqlite:///data2.sqlite')
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [9]:
coin_symbols = ["BTC","LTC","USDT","ADA","ETH"]


In [10]:

for symbol in coin_symbols:
    session.add(Coin(coin_name=symbol))
    add_df_to_table_please(session,entry,symbol)


In [11]:
loop = session.query(Entry).first()

In [12]:
# for i in loop:
    # print(i.volume)
int.from_bytes(loop.volume, byteorder="little")

8036550

In [13]:
session.close()